# LDA Demonstration-BBC News Dataset¶


In this notebook, we will build LDA models on various datasets. We'll use the gensim implementation of LDA, though sklearn also comes with one. 

We will also use the library spacy for preprocessing (specifically lemmatisation). Though you can also perform lemmatisation in NLTK, it is slightly more convenient and less verbose in spacy. For visualising the topics and the word-topic distributions (interactively!), we'll use the 'pyLDAvis' module.

In [2]:
# import libraries  
import numpy as np
import pandas as pd
import nltk
import re, random, os
import string, pprint
import matplotlib.pyplot as plt
import seaborn as sns

# spacy for basic preprocessing, optional, can use nltk as well (lemmatisation etc.)
import spacy

# gensim for LDA 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# create list of stop words
# string.punctuation (from the 'string' module) contains a list of punctuations
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + list(string.punctuation)


In [4]:
# compute coherence value at various values of alpha and num_topics
def compute_coherence_values(dictionary, corpus, texts, num_topics_range, alpha_range):
    
    coherence_values = []
    model_list = []
    for alpha in alpha_range:
        for num_topics in num_topics_range:
            lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=num_topics, 
                                               alpha=alpha,
                                               per_word_topics=True)
            model_list.append(lda_model)
            coherencemodel = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append((alpha, num_topics, coherencemodel.get_coherence()))
        

    return model_list, coherence_values

### BBC News Dataset

In [6]:
# reading the BBC news dataset

data_folder = "/mnt/c/Users/yamshee.fatima/Downloads/BBC-News-Classification-master/BBC-News-Classification-master/bbc"

folders = ["business","entertainment","politics","sport"]

x = []
y = []

for i in folders:
    files = os.listdir(data_folder + "/" + i)
    for text_file in files:
        file_path =  data_folder + "/" + i + "/" + text_file
        with open(file_path, 'rb') as f:
            data = f.read()
#         data = ' '.join(data)
        x.append(data)
        y.append(i)


data = {'news': x, 'type': y}       
df = pd.DataFrame(data)
df.to_csv('bbc_data.csv', index=False)

In [7]:
df.head()

,news,type
0,b'Ad sales boost Time Warner profit\n\nQuarter...,business
1,b'Dollar gains on Greenspan speech\n\nThe doll...,business
2,b'Yukos unit buyer faces loan claim\n\nThe own...,business
3,b'High fuel prices hit BA\'s profits\n\nBritis...,business
4,"b""Pernod takeover talk lifts Domecq\n\nShares ...",business


In [8]:
df.type.astype('category').value_counts()

business         510
sport            481
politics         417
entertainment    386
Name: type, dtype: int64

In [9]:
# filter business articles
df = df[df['type']=='business']
df.shape

(510, 2)

In [11]:
# tokenize using gensim simple_preprocess
def sent_to_words(sentences, deacc=True): # deacc=True removes punctuations
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  

# functions for removing stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [12]:
# convert to list
data = df['news'].values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words_nostops = remove_stopwords(data_words)

# lemmatize
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# create dictionary
id2word = corpora.Dictionary(data_lemmatized)

# create corpus
texts = data_lemmatized

# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4158117400316558


In [17]:
lda_model.print_topics()

[(0,
  '0.049*"hike" + 0.024*"rebound" + 0.024*"exporter" + 0.015*"seasonally" + 0.010*"straight" + 0.006*"naccording" + 0.005*"stagnant" + 0.004*"subsidise" + 0.003*"tenth" + 0.002*"stubbornly"'),
 (1,
  '0.031*"company" + 0.025*"former" + 0.021*"executive" + 0.020*"trial" + 0.019*"say" + 0.018*"charge" + 0.017*"firm" + 0.015*"fraud" + 0.015*"face" + 0.014*"claim"'),
 (2,
  '0.075*"airline" + 0.036*"retailer" + 0.030*"vital" + 0.029*"carrier" + 0.027*"higher" + 0.026*"fuel" + 0.026*"aircraft" + 0.023*"plane" + 0.018*"air" + 0.017*"joint"'),
 (3,
  '0.111*"card" + 0.003*"issuer" + 0.000*"plastic" + 0.000*"pin" + 0.000*"apacs" + 0.000*"chip" + 0.000*"debit" + 0.000*"shopper" + 0.000*"fraudster" + 0.000*"liquidation"'),
 (4,
  '0.049*"payout" + 0.041*"evidence" + 0.039*"board" + 0.030*"schedule" + 0.028*"oblige" + 0.027*"team" + 0.023*"wealth" + 0.021*"flight" + 0.020*"service" + 0.017*"passenger"'),
 (5,
  '0.039*"say" + 0.018*"country" + 0.016*"new" + 0.014*"year" + 0.012*"cost" + 0.01

In [18]:
# print keywords 
doc_lda = lda_model[corpus]

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/yamshee/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.301475 -0.119677       1        1  25.341692
5     -0.283480 -0.072636       2        1  22.128145
17    -0.300343  0.195131       3        1  17.244541
19    -0.247570  0.174133       4        1  10.740002
12    -0.221343 -0.038465       5        1   7.456028
1     -0.179552 -0.304584       6        1   6.364815
14    -0.123894  0.108808       7        1   2.761732
13     0.035226  0.054293       8        1   1.428294
18     0.051285  0.076888       9        1   1.425032
9      0.106609 -0.016272      10        1   1.325328
2      0.131113  0.007401      11        1   1.153106
4      0.133221 -0.001002      12        1   1.037785
6      0.144737 -0.003566      13        1   0.761444
16     0.154414 -0.007403      14        1   0.200730
10     0.152578 -0.009127      15        1   0.146165
15     0.151931 -0.009221      16        1   0.136271
0      0.152362 -0.009211      17        1   0.106884
3      0.149699 -0.009533      18        1   0.097322
11     0.146399 -0.006994      19        1   0.076326
7      0.148084 -0.008963      20        1   0.068347, topic_info=     Category         Freq          Term        Total  loglift  logprob
106   Default   346.000000          sale   346.000000  30.0000  30.0000
203   Default   336.000000         price   336.000000  29.0000  29.0000
93    Default   224.000000        profit   224.000000  28.0000  28.0000
275   Default   222.000000           oil   222.000000  27.0000  27.0000
21    Default   773.000000       company   773.000000  26.0000  26.0000
130   Default   925.000000          year   925.000000  25.0000  25.0000
318   Default   300.000000          cost   300.000000  24.0000  24.0000
105   Default   419.000000          rise   419.000000  23.0000  23.0000
41    Default   664.000000          firm   664.000000  22.0000  22.0000
426   Default   399.000000       economy   399.000000  21.0000  21.0000
98    Default   312.000000        report   312.000000  20.0000  20.0000
96    Default   200.000000       quarter   200.000000  19.0000  19.0000
51    Default   395.000000        growth   395.000000  18.0000  18.0000
644   Default   241.000000         state   241.000000  17.0000  17.0000
359   Default   339.000000         share   339.000000  16.0000  16.0000
1282  Default   111.000000           gas   111.000000  15.0000  15.0000
167   Default   322.000000    government   322.000000  14.0000  14.0000
206   Default   324.000000          rate   324.000000  13.0000  13.0000
425   Default   327.000000      economic   327.000000  12.0000  12.0000
35    Default   293.000000     executive   293.000000  11.0000  11.0000
110   Default   144.000000       service   144.000000  10.0000  10.0000
56    Default   230.000000      increase   230.000000   9.0000   9.0000
429   Default   271.000000        figure   271.000000   8.0000   8.0000
107   Default  2147.000000           say  2147.000000   7.0000   7.0000
69    Default   475.000000        market   475.000000   6.0000   6.0000
20    Default   115.000000         close   115.000000   5.0000   5.0000
728   Default    96.000000           car    96.000000   4.0000   4.0000
73    Default   372.000000         month   372.000000   3.0000   3.0000
1401  Default    57.000000      contract    57.000000   2.0000   2.0000
258   Default   129.000000        former   129.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
3040  Topic20     0.007396  ratification     0.725630   2.7022  -8.8122
292   Topic20     0.007396       seizure     0.725632   2.7022  -8.8122
3035  Topic20     0.007396        fellow     0.725631   2.7022  -8.8122
3045  Topic20     0.007396     signature     0.725631   2.7022  -8.8122
3032  Topic20     0.007396          bind     0.725631   2.7022  -8.8122
3044  Topic20     0.007396    shareholde     0.725632   2.7022  -8.8122
3038  Topic20     0.007396       